In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_vertexai import VertexAIEmbeddings
from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.vectorstores import BigQueryVectorSearch

from google.cloud import bigquery
from vertexai.preview.generative_models import GenerativeModel

import re

In [2]:
loader = PyPDFLoader('ontario_rent_act.pdf')
docs = loader.load_and_split()

In [6]:
len(docs)

158

In [15]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 10,
    length_function = len,
    is_separator_regex = False,
)

In [8]:
texts = text_splitter.create_documents([each.page_content for each in docs])

In [16]:
len(texts)

585

In [21]:
REGION = "US" 
DATASET = "vector_db"
TABLE = "on_vector_tbl"
PROJECT_ID = "image-genai"


In [19]:
embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)

In [20]:
client = bigquery.Client(project=PROJECT_ID, location=REGION)
client.create_dataset(dataset=DATASET, exists_ok=True)

Dataset(DatasetReference('image-genai', 'vector_db'))

In [22]:
store = BigQueryVectorSearch(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
    distance_strategy=DistanceStrategy.COSINE,
)

In [23]:
data = [t.page_content for t in texts]
metadatas = [{"len": len(t.page_content)} for t in texts]

ids = store.add_texts(data, metadatas=metadatas)

print(len(ids))

585


In [24]:
query = "Where is act being applied?"
query_vector = embedding.embed_query(query)
docs = store.similarity_search_by_vector(query_vector, k=3)
print(docs)

[Document(page_content='Provisions conflicting with Act void  \n4 (1)  Subject to subsection 12.1 (11) and section 194 , a provision in a tenancy agreement that is inconsistent with this Act or \nthe regul ations is void.  2006, c.  17, s.  4; 2017, c. 13, s. 1.  \nSame, Part V.1  \n(2)  Subject to section 194 , in any proceeding under Part V.1, a provision in an occupancy agreement in respect of a me mber \nunit or a provision in a by -law of a non -profit housing co -operative that is inconsistent with Part V.1, or with a provision in \nanother Part of this Act that applies to non -profit housing co -operatives and member units, does not apply, and the prov ision \nin this Act applies.  2013, c.  3, s. 23.  \nSection Amendments with date in force  (d/m/y)  \n2013, c. 3, s. 23  - 01/06/2014  \n2017, c. 13, s. 1  - 30/05/2017  \nExemptions from Act  \n5 This Act does not apply with respect to,  \n (a) living accommodation intended to be provided to the travelling or vacationing public 

In [25]:
def augment_prompt(query: str):
    query_vector = embedding.embed_query(query)
    # get top 3 results from knowledge base
    results = store.similarity_search_by_vector(query_vector, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the question in detail.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

In [39]:
def generate(query):
    prompt = augment_prompt(query)
    model = GenerativeModel("gemini-1.0-pro-vision-001")
    responses = model.generate_content(
        prompt,
        generation_config={
            "max_output_tokens": 2048,
            "temperature": 0.7,
            "top_p": 1
        },
        stream=False,
    )

    res = []
    print(responses.text)

    # for response in responses:
    #     print(response)
    #     # print(response.text, end="")

In [27]:
generate("Where is act being applied?")

The act applies to tenancy agreements and occupancy agreements in respect of member units or non-profit housing co-operatives.

In [28]:
generate("What is this act about?")

Based on the provided contexts, this Act primarily focuses on matters related to residential tenancies, including the establishment of rules and regulations governing the relationships between landlords and tenants. It covers various aspects of tenancy, such as the selection of tenants, provision of information by landlords, terms of tenancy agreements, resolution of disputes, and the rights and responsibilities of both parties. The Act aims to protect the interests of both landlords and tenants and ensure fair and equitable practices in residential rental arrangements.

In [40]:
generate("Who seems to be benefitting from this act, landlord or the tenants?")

Based on the provided contexts, it seems that tenants are benefitting from this act. The act provides remedies for tenants who have been induced by the landlord to vacate the rental unit, including compensation for increased rent and reasonable moving expenses. The act also places the onus on the landlord to prove that the tenant is not entitled to the remedies provided by the act.


In [41]:
generate("What are some key rights which with I can protect myself as a tenant")

**Key Rights to Protect Yourself as a Tenant:**

* **Landlord's Responsibility to Repair:** Landlords are responsible for maintaining the rental unit in a habitable condition, including repairing any necessary fixtures or appliances. (Section 20)

* **Landlord's Responsibility for Services:** Landlords are responsible for providing essential services such as heat, water, and electricity. (Section 21)

* **Landlord Not to Interfere with Reasonable Enjoyment:** Landlords cannot unreasonably interfere with your use and enjoyment of the rental unit. (Section 22)

* **Landlord Not to Harass, etc.:** Landlords are prohibited from harassing, obstructing, or coercing tenants. (Section 23)

* **Privacy:** Landlords cannot enter the rental unit without notice, except in emergency situations. (Section 25)

* **Entry with Notice:** Landlords must provide reasonable notice before entering the rental unit for repairs or other necessary purposes. (Section 27)

* **Eviction with Termination Order:** L